In [74]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime
    
base_url = 'https://www.thestage.co.uk/wp-admin/admin-ajax.php'
headers = { 'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}
reviews=[]

In [75]:
for page in range(1,130):
    
    if page%5==0:
        print(page)
        
    payload = {
        'action': 'td_ajax_block',
        'td_atts': '{"category_id":10414,"limit":"12","ajax_pagination":"load_more","post_type":["post","reviews"]}',
        'td_cur_cat': 10414,
        'td_block_id': 'td_uid_14_5b3f628908134',
        'td_column_number': 3,
        'td_current_page': page,
        'block_type': 'td_block_8_category'
    }

    session  = requests.session()
    r        = requests.post(base_url, data=payload, headers=headers)
    soup     = BeautifulSoup(r.json()['td_data'], "html5lib")
    listings = soup.find_all(attrs={"class": "span4"})

    for listing in listings:
        review = []
        title  = listing.find(attrs={"class": "entry-title"}).get_text().strip()
        url    = listing.find(attrs={"class": "entry-title"}).find('a').get('href')
        work   = title.split(' review at ')[0]
        try:
            venue = title.split(' review at ')[1].split(',')[0]        
        except:
            venue = ''
        city   = title.split(' – ')[0].split(',')[-1].strip()
        rating = listing.find(attrs={"class": "stars_small_two"}).get('data-rating').strip()
        date   = listing.find(itemprop="dateCreated").get('datetime')[0:10]
        date   = datetime.strptime(date, '%Y-%m-%d').date()         
        author = listing.find(itemprop="author").get('content')        
        
        r     = requests.get(url)
        soup  = BeautifulSoup(r.text, "html5lib")        
        text  = soup.find('div', attrs={"class": "wpb_wrapper"}).find_all('p')
        full_text=''
        for w in text:
            if w.get('class')==None:
                full_text += w.get_text()        
        
        review.append(title)
        review.append(url)        
        review.append(work)
        review.append(venue)
        review.append(city)        
        review.append(rating)        
        review.append(date)
        review.append(author)
        review.append(full_text) 
        reviews.append(review)

import pandas as pd                     
headers = ['title', 'url', 'work', 'venue', 'city', 'rating', 'date', 'author', 'full_text']
df = pd.DataFrame(reviews, columns=headers)
df.to_csv('../../data/processed/reviews/thestage.csv')        

5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
